In [ ]:
%load_ext autoreload
%autoreload 2

%config IPCompleter.greedy=True

# Stellar mass vs. $v_\text{max}$

First, imports:

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import h5py
from astropy import units
from pathlib import Path
import os

import snapshot_obj
import curve_fit
import dataset_compute

import importlib

In [ ]:
importlib.reload(snapshot_obj)
importlib.reload(curve_fit)
importlib.reload(dataset_compute)

## Get data

Next, I retrieve the data. We are only interested in luminous halos. Also, there might be some spurious cases with odd values of $v_\text{max}$.

In [ ]:
# Choose galaxy:
galaxy = 'MW'

if galaxy == 'M31':
    gn=1
else:
    gn=2

In [ ]:
# Only satellites, luminous and dark:
sim_id = "V1_MR_fix"
LCDM = snapshot_obj.Snapshot(sim_id,127,"LCDM")
vmax = LCDM.get_subhalos("Vmax")
SM = LCDM.get_subhalos("Stars/Mass")
GNs = LCDM.get_subhalos("GroupNumber")
SGNs = LCDM.get_subhalos("SubGroupNumber")

# Get the subhalos of the right galaxy:
mask_lum_m31 = np.logical_and.reduce((vmax > 0, GNs == 1, SM > 0))
mask_dark_m31 = np.logical_and.reduce((vmax > 0, GNs == 1, SM == 0))
mask_lum_mw = np.logical_and.reduce((vmax > 0, GNs == 2, SM > 0))
mask_dark_mw = np.logical_and.reduce((vmax > 0, GNs == 2, SM == 0))

# Convert to proper units:
data = {"m31" : {"luminous" : [np.arange(1, np.sum(mask_lum_m31) + 1), vmax[mask_lum_m31] * units.cm.to(units.km)],
                 "dark" : [np.arange(1, np.sum(mask_dark_m31) + 1), vmax[mask_dark_m31] * units.cm.to(units.km)]},
        "mw" : {"luminous" : [np.arange(1, np.sum(mask_lum_mw) + 1), vmax[mask_lum_mw] * units.cm.to(units.km)],
                 "dark" : [np.arange(1, np.sum(mask_dark_mw) + 1), vmax[mask_dark_mw] * units.cm.to(units.km)]}}

In [ ]:
# Sort arrays in descending order:
for gal_dict in data.values():
    for dataset in gal_dict.values():
        dataset[1] = np.sort(dataset[1])[::-1]
        print(dataset)

In [ ]:
LCDM.get_halo_number()

## Plot

In [ ]:
# Set some parameters:
x_down = 7; x_up = 100
y_down = 1; y_up = 500

In [ ]:
fig, axes = plt.subplots()

# Set axes:
axes.set_xscale('log')
axes.set_yscale('log')
axes.set_xlim(x_down,x_up)
axes.set_ylim(y_down,y_up)

# Set labels:
axes.set_xlabel('$v_{\mathrm{max}}[\mathrm{km s^{-1}}]$', fontsize=16)
axes.set_ylabel('$N(>v_{\mathrm{max}})$', fontsize=16)
#axes.set_title('Stellar mass of satellites')
#axes.text(11, 2*10**9, 'isolated galaxies')
#axes.set_title('Stellar mass of isolated galaxies')

# Galaxy text:
#self.axes.set_title('Distribution of satellites as a function of $v_{max}$')
    
# Continue to left:
for gal, gal_dict in data.items():
    for lum_or_dark, dataset in gal_dict.items():
        x = np.concatenate((dataset[0],np.array([dataset[0][-1]])))
        y = np.concatenate((dataset[1],np.array([0.01])))
        if lum_or_dark == "luminous":
            col = 'black'
        else:
            col = 'gray'
        axes.plot(y, x, label="{} {}".format(gal,lum_or_dark), c=col)

plt.tight_layout()
plt.legend()

# Save figure:

filename = 'cumulDistByVmax_{}.png'.format(sim_id)

home = os.path.dirname(snapshot_obj.__file__)
path = os.path.join(home,"Figures")

# If the directory does not exist, create it
if not os.path.exists(path):
    os.makedirs(path)
plt.savefig(os.path.join(path,filename))